In [3]:
from dlsuper.nn.NNModel import NNModel
from dlsuper.nn.NNPipe import NNPipe
from dlsuper.nn.NNLayer import NNLinearLayer, NNActivator
from dlsuper.nn.Utils import softmax, softmax_derivative
import numpy as np
import os
import struct

In [4]:
def convert_to_one_hot(y, C):
    return np.eye(C)[y.reshape(-1)].T

def calculate_accuracy(model, X, Y):
    m, n = X.shape
    Y_pre = model.predict(X)
    Y_pre = Y_pre == np.max(Y_pre, axis=0, keepdims = True)
    C = np.abs(Y_pre - Y)
    error = np.sum(C) / 2
    print(1 - error / n)

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)
    return images, labels


X_train, Y_train = load_mnist(path="MNIST")
X_train = X_train.transpose()
Y_train = convert_to_one_hot(Y_train, 10)
m, n = X_train.shape
# normalization fc-layer inputs
nn_input_min = np.min(X_train, axis=1, keepdims=True)
nn_input_max = np.max(X_train, axis=1, keepdims=True)
nn_input_final = ((X_train - nn_input_min) / (nn_input_max - nn_input_min + 1e-11))
X_train = nn_input_final

linearLayer1 = NNLinearLayer(None, m, 40)
activator1 = NNActivator(linearLayer1, 'tanh', 40)
linearLayer2 = NNLinearLayer(activator1, 40, 20)
activator2 = NNActivator(linearLayer2, 'tanh', 20)
linearLayer3 = NNLinearLayer(activator2, 20, 10)
outputLayer = NNActivator(linearLayer3, 'softmax', 10)

pipe = NNPipe(linearLayer1, outputLayer)
model = NNModel(pipe, cost_type="softmax-loss")
model.fit(X_train[:, 0:200], Y_train[:, 0:200], learning_rate=0.1, iteration_count=900)

X_test, Y_test = load_mnist(path="MNIST", kind="t10k")
X_test = X_test.transpose()
Y_test = convert_to_one_hot(Y_test, 10)

calculate_accuracy(model, X_train[:, 0:200], Y_train[:, 0:200])
calculate_accuracy(model, X_test, Y_test)

print("over")

iteration0    cost:-7.323969058732362
iteration100    cost:-0.012736612658109802
iteration200    cost:-0.003021531576053581
iteration300    cost:-0.001548298416593565
iteration400    cost:-0.0009756338302401501
iteration500    cost:-0.0006654901949857906
iteration600    cost:-0.0005023211047852013
iteration700    cost:-0.00039983894414379317
iteration800    cost:-0.0003281030277609611
1.0
0.554
over
